In [1]:
import pyspark
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf,lit
from collections import namedtuple
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")
sc = pyspark.SparkContext.getOrCreate()

# Queries:
1. the percentage of canceled flights per day, throughout the entire data set
2. weekly percentages of delays that are due to weather, throughout the entire data set 
3. the percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles.
4. a weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights. The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.


# 1) "The percentage of canceled flights per day, throughout the entire data set"


In [133]:
def getPercentage(year):
    ds = sc.textFile('./BDdata/'+year+'.csv')
    dsplitted = ds.map(lambda line : line.split(","))

    head = dsplitted.take(1)[0]
    dsfiltered = dsplitted.filter(lambda x : x != head)

    dmapped = dsfiltered.map(lambda x : (tuple([int(el) for el in x[0:3]]),int(x[21])))

    date_cancelled = dmapped.reduceByKey(lambda a, b : a+b)
    total_per_date = dmapped.map(lambda x: (x[0],1)).reduceByKey(lambda a, b : a+b)

    results=date_cancelled.join(total_per_date).map(lambda x : (x[0], x[1][0]/x[1][1]))
    return results


In [ ]:
res1 = []
for i in range(1994,2009):
    res1.append(getPercentage(str(i)).collect() for i in range(1994,2009))

# 2) "Weekly percentages of delays that are due to weather, throughout the entire data set"

In [125]:
def getWeeklyWeatherPercentage(year,remainder=None):
    ds = sc.textFile('./BDdata/'+year+'.csv')
    dsplitted = ds.map(lambda line : line.split(","))
    head = dsplitted.take(1)[0]
    dsfiltered = dsplitted.filter(lambda x : x != head)
    # "Year" , "Month", "DayofMonth" -> [0:3]
    # "ArrDelay" -> 14
    # "WeatherDelay" -> 25

    d1 = dsfiltered\
        .filter(lambda x : x[14] != 'NA' and int(x[14])>=0 and x[25]!='NA')\
        .map(lambda x : ("/".join(x[0:3]), tuple(int(el) for el in [*x[0:3],x[14],x[25]])))\
        .reduceByKey(lambda x,y: (*x[0:3], x[3]+y[3],x[4]+y[4]))\
        .map(lambda x : (datetime(*x[1][0:3]).isocalendar()[1], x[1]))

    d2 = d1.filter(lambda x : x[0] is 1 and x[1][1] is 12)
        
    d1 = d1.filter(lambda x : not (x[0] is 1 and x[1][1] is 12))
    if (remainder is not None):
        d1 = d1.union(remainder)
    d1 = d1.reduceByKey(lambda x,y : (*x[0:3], x[3]+y[3], x[4]+y[4]))
    d3b = d1.map(lambda x: (x[1][0],x[0],x[1][-1]/x[1][-2]))
    res2 = sorted(d3b.collect())
    return (res2, d2)

remainder = None
res2 = []
for year in range(1994,2009):
    r, remainder = getWeeklyWeatherPercentage(str(year), remainder)
    res2 = res2+r

In [126]:
res2

[(2003, 22, 0.021759379291139726),
 (2003, 23, 0.05435799236788522),
 (2003, 24, 0.08422798224375042),
 (2003, 25, 0.055541387509821634),
 (2003, 26, 0.026577881593444493),
 (2003, 27, 0.06815821400768812),
 (2003, 28, 0.06584921521083144),
 (2003, 29, 0.03957742490221589),
 (2003, 30, 0.07160054816598709),
 (2003, 31, 0.07311418046852729),
 (2003, 32, 0.05751709725001834),
 (2003, 33, 0.056138471877398725),
 (2003, 34, 0.060240938882279056),
 (2003, 35, 0.08129548078477976),
 (2003, 36, 0.03531433928850461),
 (2003, 37, 0.05845302266562089),
 (2003, 38, 0.036452499395229015),
 (2003, 39, 0.036826458565589),
 (2003, 40, 0.023060051550517907),
 (2003, 41, 0.04255098964109221),
 (2003, 42, 0.022278921398558266),
 (2003, 43, 0.038758046486528155),
 (2003, 44, 0.02141802677803845),
 (2003, 45, 0.04966692287001907),
 (2003, 46, 0.03960662028545622),
 (2003, 47, 0.06673508556805459),
 (2003, 48, 0.026812950851305802),
 (2003, 49, 0.06408894327539447),
 (2003, 50, 0.05878951403315525),
 (2003

# 3) "The percentage of flights belonging to a given "distance group" that were able to halve their departure delays by the time they arrived at their destinations. 
Distance groups assort flights by their total distance in miles. Flights with distances that are less than 200 miles belong in group 1, flights with distances that are between 200 and 399 miles belong in group 2, flights with distances that are between 400 and 599 miles belong in group 3, and so on. The last group contains flights whose distances are between 2400 and 2599 miles."

In [87]:
ds = sc.textFile('./BDdata/2005.csv')
dsplitted = ds.map(lambda line : line.split(","))
head = dsplitted.take(1)[0]
# head.index("Cancelled") # -> 21
# head.index("DepDelay") # -> 15
# head.index("ArrDelay") # -> 14
# head.index("Distance") # -> 18
dsfiltered = dsplitted.filter(lambda x : x != head)

d1 = dsfiltered\
    .filter(lambda x : x[21] == '0' and x[14] != 'NA' and x[15] != 'NA')\
    .map(lambda x : (x[15], x[14], x[18]))\

d2 = d1.map(lambda x : (int(x[2])//200+1,(int(x[0]),int(x[1]))))

d3 = d2.filter(lambda x : x[1][0]>=2*x[1][1])\
    .map(lambda x: (x[0],1))\
    .reduceByKey(lambda x, y : x+y)
    
d4 = d2.map(lambda x : (x[0],1))\
    .reduceByKey(lambda x, y : x+y)


d5 = d4.join(d3).map(lambda x : (x[0], x[1][1]/x[1][0]))

In [131]:
def getDistanceGroup(start,end):
    ds = sc.textFile('./BDdata/'+str(start)+'.csv')

    for i in range(start+1,end):
        ds = ds.union(sc.textFile('./BDdata/'+str(i)+'.csv'))
        
    dsplitted = ds.map(lambda line : line.split(","))
    head = dsplitted.take(1)[0]
    dsfiltered = dsplitted.filter(lambda x : x != head)

    d1 = dsfiltered\
        .filter(lambda x : x[21] == '0' and x[14] != 'NA' and x[15] != 'NA' and x[18] != 'NA')\
        .map(lambda x : (x[15], x[14], x[18]))\

    d2 = d1.map(lambda x : (int(x[2])//200+1,(int(x[0]),int(x[1]))))

    d3 = d2.filter(lambda x : x[1][0]>=2*x[1][1])\
        .map(lambda x: (x[0],1))\
        .reduceByKey(lambda x, y : x+y)
    
    d4 = d2.map(lambda x : (x[0],1))\
        .reduceByKey(lambda x, y : x+y)


    d5 = d4.join(d3).map(lambda x : (x[0], x[1][1]/x[1][0]))
    return d5
    
res3 = getDistanceGroup(1994,2009).collect()

In [132]:
sorted(res3)

[(1, 0.5644837414094587),
 (2, 0.5524500162522421),
 (3, 0.5423182678162718),
 (4, 0.545430689400957),
 (5, 0.547017139188874),
 (6, 0.5480111283552228),
 (7, 0.5521951401837523),
 (8, 0.5402587095736161),
 (9, 0.5460214402108845),
 (10, 0.5448116153770951),
 (11, 0.5343496346912973),
 (12, 0.5555007729540646),
 (13, 0.5466533748475082),
 (14, 0.5699809774656132),
 (15, 0.5610202964760749),
 (16, 0.4393708777270421),
 (17, 0.546841334137723),
 (18, 0.5037000493339912),
 (19, 0.576649060312001),
 (20, 0.5309648117492254),
 (21, 0.5611860684028868),
 (22, 0.5928038206519219),
 (23, 0.5369598443796026),
 (25, 0.578656853725851)]

# 4) A weekly "penalty" score for each airport that depends on both the its incoming and outgoing flights.
The score adds 0.5 for each incoming flight that is more than 15 minutes late, and 1 for each outgoing flight that is more than 15 minutes late.

In [111]:
ds = sc.textFile('./BDdata/2003.csv')
dsplitted = ds.map(lambda line : line.split(","))
head = dsplitted.take(1)[0]
# head.index("Cancelled") # -> 21
# head.index("DepDelay") # -> 15
# head.index("ArrDelay") # -> 14
# head.index("Distance") # -> 18
dsfiltered = dsplitted.filter(lambda x : x != head)

In [121]:
d1 = dsfiltered.filter(lambda x : x[14] != "NA" and int(x[14])>15)
d2 = d1.flatMap(lambda x : [((x[1], datetime(*[int(el) for el in x[0:3]]).isocalendar()[1], x[16]),1),
                            ((x[1], datetime(*[int(el) for el in x[0:3]]).isocalendar()[1], x[17]),0.5)])
d2a = d2.filter(lambda x : x[0][1] == "12" and x[0][2] == 1)
d2b = d2.filter(lambda x : not(x[0][1] == "12" and x[0][2] == 1))
d3 = d2b.map(lambda x : ((x[0][2],x[0][1]),x[1])).reduceByKey(lambda x, y : x+y)
res = {"2003":d3.collect()}


In [136]:
def getWeeklyPenalty(year, remainder):
    ds = sc.textFile('./BDdata/'+str(year)+'.csv')
    dsplitted = ds.map(lambda line : line.split(","))
    head = dsplitted.take(1)[0]

    dsfiltered = dsplitted.filter(lambda x : x != head)
    d1 = dsfiltered.filter(lambda x : x[14] != "NA" and int(x[14])>15)
    d2 = d1.flatMap(lambda x : [((x[1], datetime(*[int(el) for el in x[0:3]]).isocalendar()[1], x[16]),1),
                                ((x[1], datetime(*[int(el) for el in x[0:3]]).isocalendar()[1], x[17]),0.5)])
    d2a = d2.filter(lambda x : x[0][1] == "12" and x[0][2] == 1)
    d2b = d2.filter(lambda x : not(x[0][1] == "12" and x[0][2] == 1))
    d3 = d2b.map(lambda x : ((year,x[0][2],x[0][1],x[0][2]),x[1])).reduceByKey(lambda x, y : x+y)
    return d3.collect()

remainder = None
res3 = []
for year in range(1994,2009):
    r, remainder = getWeeklyWeatherPercentage(str(year), remainder)
    res3 = res3+r

In [137]:
res3

[(2003, 22, 0.021759379291139726),
 (2003, 23, 0.05435799236788522),
 (2003, 24, 0.08422798224375042),
 (2003, 25, 0.055541387509821634),
 (2003, 26, 0.026577881593444493),
 (2003, 27, 0.06815821400768812),
 (2003, 28, 0.06584921521083144),
 (2003, 29, 0.03957742490221589),
 (2003, 30, 0.07160054816598709),
 (2003, 31, 0.07311418046852729),
 (2003, 32, 0.05751709725001834),
 (2003, 33, 0.056138471877398725),
 (2003, 34, 0.060240938882279056),
 (2003, 35, 0.08129548078477976),
 (2003, 36, 0.03531433928850461),
 (2003, 37, 0.05845302266562089),
 (2003, 38, 0.036452499395229015),
 (2003, 39, 0.036826458565589),
 (2003, 40, 0.023060051550517907),
 (2003, 41, 0.04255098964109221),
 (2003, 42, 0.022278921398558266),
 (2003, 43, 0.038758046486528155),
 (2003, 44, 0.02141802677803845),
 (2003, 45, 0.04966692287001907),
 (2003, 46, 0.03960662028545622),
 (2003, 47, 0.06673508556805459),
 (2003, 48, 0.026812950851305802),
 (2003, 49, 0.06408894327539447),
 (2003, 50, 0.05878951403315525),
 (2003